# Processing the final dataset



In [4]:
import ee
import geemap

# Authenticate to Earth Engine
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

data_folder = "projects/amazon-forest-regrowth/assets"


In [5]:
export_distances = False
aggregate_LU = False

if (aggregate_LU):
    prefix = "aggregated"
else:
    prefix = "non_aggregated"

In [6]:
mature_biomass = ee.Image(f"{data_folder}/mature_biomass_500m")
proj = mature_biomass.projection().getInfo()

age_agbd = ee.Image(f"{data_folder}/age_agbd")
secondary = age_agbd.select("age")

if (export_distances):
    mature_biomass_vector = mature_biomass.reduceToVectors(geometry = secondary.geometry(), scale=500, maxPixels=1e13)
    distances_image = mature_biomass_vector.distance(searchRadius = 500000).toInt().updateMask(secondary)

    # Export the mean_biomass_image as an asset
    task = ee.batch.Export.image.toAsset(
        image=distances_image,
        assetId = "projects/amazon-forest-regrowth/assets/distance_to_mature_biomass",
        description="distance_to_mature_biomass",
        region=secondary.geometry(),
        crs=proj['crs'],
        crsTransform=proj['transform'],
        maxPixels=1e13
    )
    task.start()



In [7]:

# Function to buffer and calculate mean
def buffer_distance(f):
    distance = ee.Number(f.get('distance'))
    buffered = ee.Feature(f.buffer(distance.add(2000)))
    return buffered


In [8]:
categorical = ee.Image(f"{data_folder}/categorical")
distance = ee.Image(f"{data_folder}/distance_to_mature_biomass")
one_hectare_mask = ee.Image(f"{data_folder}/one_hectare_mask")

# Create masks for biomes 1 and 4
biome_mask = categorical.select('biome').eq(1).Or(categorical.select('biome').eq(4)).Or(categorical.select('biome').eq(6))
combined_mask = one_hectare_mask.And(biome_mask)
categorical = categorical.updateMask(combined_mask)
dist_cat = distance.addBands(combined_mask)


In [9]:
nearest_mature = ee.FeatureCollection("projects/amazon-forest-regrowth/assets/nearest_mature")
# change geometries from polygons to points
nearest_mature = nearest_mature.map(lambda f: ee.Feature(f.geometry().centroid()).copyProperties(f))

# Convert the FeatureCollection to an image
nearest_mature_image = nearest_mature.reduceToImage(
    properties=['first'],
    reducer=ee.Reducer.first()
).toInt16().rename('nearest_mature')

export_task = ee.batch.Export.image.toAsset(
    image=nearest_mature_image,
    assetId = "projects/amazon-forest-regrowth/assets/nearest_mature_image",
    description='nearest_mature_export',
    crs=proj['crs'],
    crsTransform=proj['transform'],
    region = age_agbd.geometry(),
    maxPixels=1e13
)

# export_task.start()

In [10]:

def export_csv(year, prefix):

    if (year != 0):
        prefix += f"_{year}yr"

    # Load the image collections
    Transition = ee.ImageCollection('projects/JRC/TMF/v1_2023/TransitionMap_Subtypes').mosaic().clip(age.geometry())
    AnnualChanges = ee.ImageCollection('projects/JRC/TMF/v1_2023/AnnualChanges').mosaic().clip(age.geometry())

    # Define regrowth and degraded conditions
    regrowth = Transition.gte(31).And(Transition.lte(33))  # include classes 63, 64 if you want to add regrowing mangroves

    # Initialize AgeRegrowth and AgeDegraded
    AgeRegrowth = ee.Image.constant(0)

    # Calculate AgeRegrowth
    for i in range(1990, 2024):
        year = 'Dec' + str(i)
        AnnualChangesYear = AnnualChanges.select(year)
        condition = AnnualChangesYear.eq(4).And(regrowth)
        AgeRegrowth = AgeRegrowth.add(condition.eq(1))

    # land_use_years = ee.Image(f"{data_folder}/land_use_{prefix}")
    # nearest_mature = ee.Image(f"{data_folder}/nearest_mature_image").toInt16()
    # yearly_mean_prec = ee.Image(f"{data_folder}/yearly_mean_prec")
    # yearly_SI = ee.Image(f"{data_folder}/yearly_SI")
    cwd = ee.Image(f"{data_folder}/raw/cwd_chave")
    # sur_cover = ee.Image(f"{data_folder}/sur_cover").rename("sur_cover")

    # combined_mask = land_use_years.select("last_LU").And(nearest_mature)

    # unified_data = (
    #     age_agbd.addBands(
    #         [distance, nearest_mature, categorical, land_use_years, cwd, sur_cover, yearly_mean_prec, yearly_SI
    #     ]).updateMask(combined_mask)
    # )

    unified_data = (
        AgeRegrowth.selfMask().addBands(cwd)
    )

    region = unified_data.geometry()
    grid = region.coveringGrid(region.projection(), 1000)

    # Function to check if there's at least one cell of unified_data in the grid cell
    def check_overlap(grid_cell):
        geometry = grid_cell.geometry()
        
        sampled = unified_data.stratifiedSample(
            numPoints = 1, classBand = 'biome', region = geometry, scale=50000#, geometries = True
        )

        return sampled.first()


    # Apply the function to each grid cell
    unified_data_sampled = grid.map(check_overlap, dropNulls = True)

    # # Define the list of bands to check
    # bands_to_check = ['lulc_sum_20', 'lulc_sum_21', 'lulc_sum_35', 
    #                 'lulc_sum_39', 'lulc_sum_40', 'lulc_sum_41', 
    #                 'lulc_sum_46', 'lulc_sum_48', 'lulc_sum_9']

    # # Create a mask where at least one of the specified bands is non-zero
    # mask = unified_data.select(bands_to_check).reduce(ee.Reducer.anyNonZero())

    # # # Apply the mask to the unified_data
    # unified_data = unified_data.updateMask(mask)

    # unified_data_sampled = unified_data.stratifiedSample(
    #     numPoints = 10000, classBand = 'biome', region = age_agbd.geometry()
    # )
    
    # export_name = f"{prefix}_neigh_reduced"

    task = ee.batch.Export.table.toDrive(
        collection = unified_data_sampled, description = prefix, fileFormat = 'CSV'
    )
    task.start()

    # task = ee.batch.Export.table.toAsset(
    #     collection = unified_data_sampled, description = f"{prefix}_toAsset", assetId=f"projects/amazon-forest-regrowth/assets/{prefix}"
    # )
    # task.start()

    return unified_data

unified_data = export_csv(0, prefix)
# export_csv(5, prefix)
# export_csv(10, prefix)
# export_csv(15, prefix)

In [18]:
year = 5
if (year != 0):
    prefix += f"_{year}yr"

land_use_years = ee.Image(f"{data_folder}/land_use_{prefix}")
nearest_mature = ee.Image(f"{data_folder}/nearest_mature_image").toInt16()
yearly_mean_prec = ee.Image(f"{data_folder}/yearly_mean_prec")
yearly_SI = ee.Image(f"{data_folder}/yearly_SI")
cwd = ee.Image(f"{data_folder}/raw/cwd_chave")
sur_cover = ee.Image(f"{data_folder}/sur_cover").rename("sur_cover")

combined_mask = land_use_years.select("last_LU").And(nearest_mature)

unified_data = (
    age_agbd.addBands(
        [distance, nearest_mature, categorical, land_use_years, cwd, sur_cover, yearly_mean_prec, yearly_SI
    ]).updateMask(combined_mask)
)

# unified_data = unified_data.reduceNeighborhood(
#     ee.Reducer.firstNonNull(),
#     ee.Kernel.square(radius = 500)
# )

# Define the list of bands to check
bands_to_check = ['lulc_sum_20', 'lulc_sum_21', 'lulc_sum_35', 
                'lulc_sum_39', 'lulc_sum_40', 'lulc_sum_41', 
                'lulc_sum_46', 'lulc_sum_48', 'lulc_sum_9']

# Create a mask where at least one of the specified bands is non-zero
mask = unified_data.select(bands_to_check).reduce(ee.Reducer.anyNonZero())

# # Apply the mask to the unified_data
unified_data = unified_data.updateMask(mask)

map = geemap.Map()
map.addLayer(unified_data, {}, "unified_data")
map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [11]:
gaul = ee.FeatureCollection('FAO/GAUL_SIMPLIFIED_500m/2015/level0')
br_shp = ee.FeatureCollection('projects/amazon-forest-regrowth/assets/raw/brasil_shp')

# Convert the FeatureCollection to an image
br_img = br_shp.reduceToImage(
    properties=['AREA_KM2'],  # Assuming 'first' is the current property name
    reducer=ee.Reducer.first()
).rename('br_img')

br_img = br_img.reproject(proj['crs'], proj['transform'])
br_img = br_img.gt(0)


south_american_countries = [
    'Argentina', 'Bolivia', 'Brazil', 'Chile', 'Colombia', 'Ecuador',
    'French Guiana', 'Guyana', 'Paraguay', 'Peru', 'Suriname', 'Uruguay', 'Venezuela'
]

south_america = gaul.filter(ee.Filter.inList('ADM0_NAME', south_american_countries))

south_america_merged = south_america.union().geometry()

br_buffer = br_shp.geometry().buffer(-1000, maxError=5)
sa_buffer = south_america_merged.buffer(-1000, maxError=5)

remove_mask = br_buffer.symmetricDifference(sa_buffer)

remove_mask = ee.FeatureCollection(ee.Feature(remove_mask))

# Export the mean_biomass_image as an asset
task = ee.batch.Export.table.toAsset(
    collection=remove_mask,
    assetId = "projects/amazon-forest-regrowth/assets/remove_mask",
    description="remove_mask"
)
task.start()

# map = geemap.Map()
# map.addLayer(br_img, {}, "br_img")
# map.addLayer(brazil_buffer, {}, "brazil_buffer")
# # map.addLayer(south_america_merged, {}, "largest_polygon")
# map

In [57]:

# Convert the MultiPolygon to a FeatureCollection of individual polygons
geometry_fc = ee.FeatureCollection(south_america_merged.geometries().map(lambda geom: ee.Feature(ee.Geometry(geom))))

# Calculate the area of each polygon and add it as a property
area_fc = geometry_fc.map(lambda f: f.set('area', f.geometry().area()))

# Sort the polygons by area in descending order
sorted_fc = area_fc.sort('area', False)

# Get the largest polygon (the first feature in the sorted collection)
largest_polygon = ee.Feature(sorted_fc.first()).geometry()
